In [34]:
from langchain_ollama import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

from langchain_core.documents import Document
from langchain_chroma import Chroma

import uuid
from langchain_community.document_loaders import PyPDFLoader

In [35]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=510, 
    chunk_overlap=50,
    length_function=len,
    separators=["\n", ".", "\n\n"]
)

In [36]:
# Probar distintos modelos de embeddings
embeddings_generator = OllamaEmbeddings(
    model='mxbai-embed-large:latest',
)

In [37]:
chroma_db_path = "./database/chroma_db"

vector_store = Chroma(
    persist_directory=chroma_db_path,
    embedding_function=embeddings_generator,
    collection_name="kanji"
)

In [38]:
# Documentos txt
txt_path = './database/docs/txt/kanji-1.txt'
txt_text = ''

with open(txt_path, 'r', encoding='utf-8') as file:
    txt_text = file.read()

print(len(txt_text))

12899


In [39]:
text_chunks = text_splitter.split_text(txt_text)
print(len(text_chunks))

39


In [ ]:
for chunk in text_chunks:
    document = Document(
        id=str(uuid.uuid4()),
        page_content=chunk,
        metadata={
            'name': 'kanji',
            'source': 'https://mirandohaciajapon.com/el-kanji-su-utilidad-y-funcion/',
            'generation': 6,
            'image_src': 'https://mirandohaciajapon.com/wp-content/uploads/2016/09/kanji-japon-768x574.jpg'
        }
    )

    vector_store.add_documents([document])

In [41]:
vector_store.similarity_search("Que es un kanji?", k=5)

[Document(id='ed407d8f-1326-4eb2-b908-5790181883ca', metadata={'name': 'kanji', 'generation': 6, 'image_src': 'https://upload.wikimedia.org/wikipedia/commons/thumb/9/95/Kanji_furigana.svg/163px-Kanji_furigana.svg.png', 'source': 'https://es.wikipedia.org/wiki/Kanji'}, page_content="Una sucesión de kanjis en una palabra suele dar una lectura on'yomi. Tomemos el mismo kanji del ejemplo anterior, 'ver' (見る miru?), que cuando se usa en combinación con otro, se puede leer 'ken', como en 'opinión' / 'punto de vista' (意見 iken?). Esta regla también tiene excepciones, palabras que aunque se representan con una combinación de kanjis, tienen una lectura kun'yomi. Por ejemplo, 'carta' (手紙 tegami?).\n\nLos kanjis oficiales"),
 Document(id='3138acb9-277f-4f71-9327-aee0d24de967', metadata={'image_src': 'https://upload.wikimedia.org/wikipedia/commons/thumb/9/95/Kanji_furigana.svg/163px-Kanji_furigana.svg.png', 'generation': 6, 'name': 'kanji', 'source': 'https://es.wikipedia.org/wiki/Kanji'}, page_con

In [ ]:
pdf_document = PyPDFLoader('./database/docs/PDF/KanjiParaRecordar.pdf')
pdf_pages = pdf_document.load()

In [ ]:
pdf_chunks = text_splitter.split_documents(pdf_pages)
print(len(pdf_chunks))

In [ ]:
for chunk in pdf_chunks:
    chunk.id = str(uuid.uuid4())
    chunk.metadata = {
        "name": "definicion"
    }

In [ ]:
vector_store.add_documents(pdf_chunks)

In [ ]:
vector_store.similarity_search("Que es la megaevolucion?", k=5)

In [32]:

# Para eliminar la colección
vector_store.delete_collection()
vector_store.reset_collection()

ValueError: Chroma collection not initialized. Use `reset_collection` to re-create and initialize the collection. 